## 1. First we are gonna scrape the data from Wikipedia and then clean it
---

#### Let's import pandas and numpy, then scrape the Wikipedia page tables and check how many we got

In [1]:
import pandas as pd
import numpy as np

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
len(dfs)

3

#### Choosing the table of interest

In [2]:
toronto = dfs[0]
toronto

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Let's drop the "Not assigned" values from Borough and reset the indexes

In [3]:
toronto.drop(labels = toronto[toronto['Borough'] == 'Not assigned'].index, inplace = True)
toronto.reset_index(inplace = True, drop = True)

In [4]:
print(toronto.shape)
toronto

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## 2. Now we are gonna get the latitude and longitude for each postal code
---

#### Let's import the required libraries, and download the data

In [5]:
import geocoder
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')

#### Now let's join the data downloaded with the toronto dataframe

In [6]:
toronto_ll = toronto.merge(lat_lon, on = 'Postal Code')
toronto_ll

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## 3. Finally we are gonna explore the neighborhoods using foursquare API, cluster them using the k-means algorithm, and then visualize them in a map
---

#### First let's import all the necessary packages

In [7]:
import requests
from sklearn.cluster import KMeans
import matplotlib as plt
import folium

#### Now I'll define my credentials to use the foursquare API

In [8]:
client_id = 'removed for sharing'
client_secret = 'removed for sharing'
version = '20201113'
limit = 100
radius = 500

#### Let's define the function that will get the request for each neighborhood and store the relevant info in a Dataframe to work with

In [9]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']

        for venue in results:
            venues_list.append((
                name, 
                lat, 
                lng, 
                venue['venue']['name'], 
                venue['venue']['location']['lat'], 
                venue['venue']['location']['lng'],  
                venue['venue']['categories'][0]['name']))
    
    nearby_venues = pd.DataFrame(venues_list, columns = ['Neighbourhood', 'Neighbourhood Latitude',                                                                       'Neighbourhood Longitude', 'Venue', 'Venue Latitude',                                                            'Venue Longitude', 'Venue Category'])
    
    return(nearby_venues)

#### Now let's use the function to get the dataframe with the venues data and explore it to get a sense of its structure

In [10]:
toronto_venues = getNearbyVenues(toronto_ll['Neighbourhood'], toronto_ll['Latitude'], toronto_ll['Longitude'])
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [11]:
print('The venues Dataframe shape is:', toronto_venues.shape)
toronto_venues[['Neighbourhood', 'Venue']].groupby('Neighbourhood').count().sort_values(by = 'Venue', ascending = False)

The venues Dataframe shape is: (2139, 7)


,Venue
Neighbourhood,
"Toronto Dominion Centre, Design Exchange",100
"Richmond, Adelaide, King",100
"Harbourfront East, Union Station, Toronto Islands",100
"Garden District, Ryerson",100
"First Canadian Place, Underground city",100
...,...
Roselawn,2
"West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale",1
"Humberlea, Emery",1


#### We see that there is a difference in the number of neighbourhoods from the last dataframe (toronto_ll), so let's check which neighbourhoods got left out because of lack of information in Foursquare

In [12]:
ll = list(toronto_ll['Neighbourhood'])
venues = list(toronto_venues['Neighbourhood'])
diff = []

for item in ll:
    if item not in venues:
        diff.append([item])
print(diff)

[['Islington Avenue, Humber Valley Village'], ['Willowdale, Newtonbrook'], ['Upper Rouge']]


#### Let's see how many different venues we have across Toronto

In [13]:
toronto_venues[['Venue Category', 'Venue']].groupby('Venue Category').count().sort_values('Venue', ascending=False)

,Venue
Venue Category,
Coffee Shop,192
Café,93
Restaurant,67
Park,51
Pizza Place,50
...,...
Flea Market,1
Fruit & Vegetable Store,1
Other Great Outdoors,1


#### Ok now we want to cluster the different neighbourhoods based on their venues's types, so let's accommodate the dataframe for said analysis

In [14]:
pd.set_option('display.max_columns', None)
toronto_dum = pd.get_dummies(toronto_venues['Venue Category'])
toronto_dum.insert(loc = 0, column = 'Neighbourhood', value = toronto_venues['Neighbourhood'] ) 
toronto_dum

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [15]:
toronto_dum = toronto_dum.groupby('Neighbourhood').mean()
toronto_dum

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

#### Now that we have the desired dataframe, let's apply the k-means algorithm to cluster the neighbourhoods

In [16]:
kclusters = 4
clusters = KMeans(n_clusters = kclusters, random_state = 0)
clusters.fit(toronto_dum)
clusters.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 3, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1])

In [17]:
toronto_clustered = toronto_dum.copy()
toronto_clustered.insert(loc = 0, column = 'Group label', value = clusters.labels_)
toronto_clustered.reset_index(inplace = True)
toronto_clustered

,Neighbourhood,Group label,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [18]:
toronto_final = toronto_clustered[['Neighbourhood', 'Group label']].merge(toronto_ll[['Neighbourhood', 'Latitude', 'Longitude']], on = 'Neighbourhood')
toronto_final

,Neighbourhood,Group label,Latitude,Longitude
0,Agincourt,1,43.794200,-79.262029
1,"Alderwood, Long Branch",1,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",1,43.754328,-79.442259
3,Bayview Village,1,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",1,43.733283,-79.419750
...,...,...,...,...
95,"Willowdale, Willowdale West",1,43.782736,-79.442259
96,Woburn,1,43.770992,-79.216917
97,Woodbine Heights,1,43.695344,-79.318389
98,York Mills West,0,43.752758,-79.400049


#### Now with this Dataframe let's create a map to visualize how the clusters ended

In [19]:
toronto_latitude = 43.6532
toronto_longitude = -79.3832

map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10.5)

colors = ['green', 'blue', 'red', 'black', 'orange', 'darkred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

for lat, lon, neigh, clust in zip(toronto_final['Latitude'], toronto_final['Longitude'],                                                            toronto_final['Neighbourhood'], toronto_final['Group label']):
    label = folium.Popup(str(neigh) + ' (Cluster ' + str(clust) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[clust],
        fill = True,
        fill_opacity = 0.7).add_to(map_clusters)

map_clusters

#### Great! To finish the analysis let's take a look at the most common venues in each neighbourhood, to get an idea of how the clusters formed

In [20]:
ntop_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood', 'Group label']
for ind in range(ntop_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

toronto_top_venues = pd.DataFrame(columns=columns)

for ind in range(toronto_clustered.shape[0]):
    row = []
    row = [toronto_clustered['Neighbourhood'][ind], toronto_clustered['Group label'][ind]]
    venues = toronto_clustered.iloc[ind, 2:].sort_values(ascending = False)[0:ntop_venues]
    for v in venues.iteritems():
        if v[1] == 0.0:
            row.append('-')
        else:
            row.append(v[0])
    toronto_top_venues.loc[ind] = row
toronto_top_venues

,Neighborhood,Group label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,1,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,-,-,-,-,-
1,"Alderwood, Long Branch",1,Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,-,-,-,-
2,"Bathurst Manor, Wilson Heights, Downsview North",1,Bank,Coffee Shop,Frozen Yogurt Shop,Chinese Restaurant,Shopping Mall,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,1,Japanese Restaurant,Café,Chinese Restaurant,Bank,-,-,-,-,-,-
4,"Bedford Park, Lawrence Manor East",1,Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Thai Restaurant,Locksmith,Liquor Store,Comfort Food Restaurant,Butcher,Juice Bar
...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",1,Pizza Place,Grocery Store,Butcher,Coffee Shop,Pharmacy,-,-,-,-,-
92,Woburn,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,-,-,-,-,-,-,-
93,Woodbine Heights,1,Skating Rink,Intersection,Dance Studio,Park,Beer Store,Athletics & Sports,Curling Ice,-,-,-
94,York Mills West,0,Park,Convenience Store,-,-,-,-,-,-,-,-


#### To finish, let's take a look at each separate cluster with its most common venues

In [21]:
for k in range(kclusters):
    print('\nThis is the group: ' + str(k), end='\n\n')
    display(toronto_top_venues[toronto_top_venues['Group label'] == k])


This is the group: 0



,Neighborhood,Group label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Caledonia-Fairbanks,0,Park,Women's Store,Pool,-,-,-,-,-,-,-
26,"East Toronto, Broadview North (Old East York)",0,Intersection,Park,Convenience Store,-,-,-,-,-,-,-
30,"Forest Hill North & West, Forest Hill Road Park",0,Trail,Park,Sushi Restaurant,Jewelry Store,-,-,-,-,-,-
46,Lawrence Park,0,Park,Swim School,Bus Line,-,-,-,-,-,-,-
50,"Milliken, Agincourt North, Steeles East, L'Amo...",0,Playground,Park,Bakery,-,-,-,-,-,-,-
54,"North Park, Maple Leaf Park, Upwood Park",0,Bakery,Park,Construction & Landscaping,-,-,-,-,-,-,-
61,Parkwoods,0,Park,Food & Drink Shop,-,-,-,-,-,-,-,-
65,Rosedale,0,Park,Playground,Trail,-,-,-,-,-,-,-
88,Weston,0,Park,-,-,-,-,-,-,-,-,-
94,York Mills West,0,Park,Convenience Store,-,-,-,-,-,-,-,-



This is the group: 1



,Neighborhood,Group label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,1,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,-,-,-,-,-
1,"Alderwood, Long Branch",1,Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,-,-,-,-
2,"Bathurst Manor, Wilson Heights, Downsview North",1,Bank,Coffee Shop,Frozen Yogurt Shop,Chinese Restaurant,Shopping Mall,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,1,Japanese Restaurant,Café,Chinese Restaurant,Bank,-,-,-,-,-,-
4,"Bedford Park, Lawrence Manor East",1,Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Thai Restaurant,Locksmith,Liquor Store,Comfort Food Restaurant,Butcher,Juice Bar
...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",1,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Japanese Restaurant,Pizza Place,Coffee Shop,Café,Movie Theater,Juice Bar,Grocery Store
91,"Willowdale, Willowdale West",1,Pizza Place,Grocery Store,Butcher,Coffee Shop,Pharmacy,-,-,-,-,-
92,Woburn,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,-,-,-,-,-,-,-
93,Woodbine Heights,1,Skating Rink,Intersection,Dance Studio,Park,Beer Store,Athletics & Sports,Curling Ice,-,-,-



This is the group: 2



,Neighborhood,Group label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,"Humberlea, Emery",2,Baseball Field,-,-,-,-,-,-,-,-,-
58,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2,Construction & Landscaping,Baseball Field,-,-,-,-,-,-,-,-



This is the group: 3



,Neighborhood,Group label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,"Malvern, Rouge",3,Fast Food Restaurant,-,-,-,-,-,-,-,-,-
